# Tutorial - Data transformations and BLAST databases


*In this tutorial we are going to perform some basic data transformations on a flat file database, convert it into a BLAST database and use it in an experiment. We will benchmark our analyses to identify what is the most appropriate tool to use. This is real data, and the results may be interesting.*

### The problem:

We have an *Arabidopsis thaliana* reporter line that contains two reporter gene constructs which have integrated in two different unknown locations. The first reporter they contain is a luciferase reporter called *CAB2:LUC+*, the promoter of the CAB2 gene fused to firefly luciferase. The second reporter they contain is the *35S:AEQ* construct, the strong 35S viral promoter driving expression of the AEQUORIN gene from jellyfish. Both of these genes make the plant glow in the dark.

We do not know the genomic locations of the inserts, however the gene for LUC and AEQ are on the end of each construct, and so should be neighbouring genomic DNA. We need to find out where the genes are integrated, and have some NGS reads from the line generated for a different experiment (Hearn et al., 2018).


### The solution:
We need to convert the FASTQ file into something that can be searched against efficiently using an algorithm. We will do this using BLAST. We need to choose a way to transform our FASTQ data into FASTA data. Then we need to make a BLAST library from our sequence FASTA data and search it using the sequence of firefly luciferase and apoaequorin. 

### The method:

We want to start in a directory that contains both FASTA_queries and FASTQ_data. There are four files to choose from. We will run them all sequentially, as they all contain different amounts of seqeunces subsampled from the same original fastq file. Choose one file to begin with. The code cells that follow are all formatted for the file called "32.fastqsanger". Note that the file extension of reads may be .fastqsanger, .fastq or simply .fq. 

In [ ]:
%%bash
cd FASTQ_data 
ls

**Step 1)** Choose a method for converting from fastq to fasta. Two different methods are presented here, <code>awk</code> and EMBOSS:seqret. You could also use <code>sed</code> or the fastx toolkit. Use one of these methods to begin with. It is always good to check whether a Unix command or a software tool developed specifically for bioinformatics is best for what we want to achieve. 

In [ ]:
%%bash
cat ~/FASTQ_data/32.fastqsanger | awk '{if(NR%4==1) {printf(">%s\n",substr($0,2));} else if(NR%4==2) print;}' > 32.fasta
head 32.fasta

In [ ]:
%%bash
seqret -sequence ~/FASTQ_data/32.fastqsanger -outseq 32.fasta
head 32.fasta

We can benchmark a process in terms of the time it takes to run. For dealing with very large FASTQ and FASTA files this is very important to do. I benchmark using the <code>perf stat</code> command. Replace the text in pointy brackets with whatever tool above you preferred. You may have a favoured alternative e.g. <code>time</code>, so please substitute as you see fit. To make running <code>perf stat</code> easier I have placed the fastq to fasta line inside a bash script.

In [ ]:
%%writefile awk_fastq.sh
#!/bin/bash
cat ~/FASTQ_data/32.fastqsanger | awk '{if(NR%4==1) {printf(">%s\n",substr($0,2));} else if(NR%4==2) print;}' > 32.fasta

In [ ]:
%%!
sudo perf stat -r 10 -d --table -o seqret_perf.txt bash ./awk_fastq.sh

**Step 2)** Make a blast library. magicblast has a command to make a blast database <code>makeblastdb</code>. Magic-blast and BLAST are pre-installed on your VMs. Note that some older information on the internet will recommend using the EMBOSS formatdb package. Please do not use this. There is a good overview here: https://www.ncbi.nlm.nih.gov/books/NBK569841/. Detailed instructions for using <code>makeblastdb</code> are available here: https://ncbi.github.io/magicblast/cook/blastdb.html

In [ ]:
%%!
makeblastdb -in 32.fasta -dbtype nucl -out 32_database

For this command we specify the following arguments:
        <code>-in </code>
        <code>-dbtype</code>
        <code>-out</code>

**Step 3)** When you have configured a BLAST library it is time to run <code>blastn</code> using one of the query sequences in the FASTA query folder. The overview in the BLAST+ user manual is useful to read before beginning: https://www.ncbi.nlm.nih.gov/books/NBK569856/. There are four query sequences in the FASTA_query directory - choose one of these to work with.

In [ ]:
%%bash
cd FASTA_queries 
ls

In [ ]:
%%!
blastn -db 32_database -query ~/FASTA_queries/firefly.fa -out results_db.out

For this command we specify the following arguments:
        <code>-db </code>
        <code>-query</code>
        <code>-out</code>

**STOP!** Why did we need to make a database for BLAST to run? BLAST will accepy a plain flat file fasta as input -try it out and see. Instead of specifying the <code>-db</code> argument use <code>-subject</code> instead.

In [ ]:
%%!
blastn -subject 32.fasta -query ~/FASTA_queries/firefly.fa -out results_fas.out

Let's identify the main reason why we need to make the blast database and index. Compare the performance of using <code>-db</code> over <code>-subject</code> with the <code>perf stat</code> command. The various arguments for <code>blastn</code> confuse <code>perf stat</code> so I have placed them in single line bash scripts again.

In [ ]:
%%writefile blast_fas.sh
#!/bin/bash
/anaconda/envs/py37_default/bin/blastn -subject 32.fasta -query ~/FASTA_queries/firefly.fa -out results_fas.out

In [ ]:
%%writefile blast_db.sh
#!/bin/bash
/anaconda/envs/py37_default/bin/blastn -db 32_database -query ~/FASTA_queries/firefly.fa -out results_db.out

In [ ]:
%%!
sudo perf stat -r 10 -d --table -o time_fs.txt bash ./blast_fas.sh

In [ ]:
%%!
sudo perf stat -r 10 -d --table -o time_db.txt bash ./blast_db.sh

Let me know what you found, and your tool preference!